<a href="https://colab.research.google.com/github/hasansin/PySpark/blob/master/PySpark_Practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

df = spark.read.csv('/content/cus.csv',header = True, inferSchema = True);

df.show(5)

+--------+--------+------+--------+----+----+----+
|order_id|customer|amount|  status| _c4| _c5| _c6|
+--------+--------+------+--------+----+----+----+
|    1001|   Alice|   200|complete|NULL|NULL|NULL|
|    1002|     Bob|  NULL| pending|NULL|NULL|NULL|
|    1003|   Alice|   350|complete|NULL|NULL|NULL|
|    1004| Charlie|   150|complete|NULL|NULL|NULL|
|    1005|     Bob|   300|complete|NULL|NULL|NULL|
+--------+--------+------+--------+----+----+----+
only showing top 5 rows



In [4]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



In [5]:
df.describe().show()

+-------+------------------+--------+------------------+---------+----+----+----+
|summary|          order_id|customer|            amount|   status| _c4| _c5| _c6|
+-------+------------------+--------+------------------+---------+----+----+----+
|  count|                10|      10|                 7|       10|   0|   0|   0|
|   mean|            1005.5|    NULL|278.57142857142856|     NULL|NULL|NULL|NULL|
| stddev|3.0276503540974917|    NULL| 118.5226520443204|     NULL|NULL|NULL|NULL|
|    min|              1001|   Alice|               150|cancelled|NULL|NULL|NULL|
|    max|              1010|    Evan|               500|  pending|NULL|NULL|NULL|
+-------+------------------+--------+------------------+---------+----+----+----+



In [8]:
df_clean = df.dropna(subset=['customer','status','amount'])
df_clean.show(10)

+--------+--------+------+--------+----+----+----+
|order_id|customer|amount|  status| _c4| _c5| _c6|
+--------+--------+------+--------+----+----+----+
|    1001|   Alice|   200|complete|NULL|NULL|NULL|
|    1003|   Alice|   350|complete|NULL|NULL|NULL|
|    1004| Charlie|   150|complete|NULL|NULL|NULL|
|    1005|     Bob|   300|complete|NULL|NULL|NULL|
|    1007| Charlie|   200| pending|NULL|NULL|NULL|
|    1008|   Diana|   500|complete|NULL|NULL|NULL|
|    1009|   Diana|   250|complete|NULL|NULL|NULL|
+--------+--------+------+--------+----+----+----+



In [13]:
df_remove_null = df_clean.select([cl for cl in df.columns if df.filter(col(cl).isNotNull()).count() > 0])

df_remove_null.show(10)

+--------+--------+------+--------+
|order_id|customer|amount|  status|
+--------+--------+------+--------+
|    1001|   Alice|   200|complete|
|    1003|   Alice|   350|complete|
|    1004| Charlie|   150|complete|
|    1005|     Bob|   300|complete|
|    1007| Charlie|   200| pending|
|    1008|   Diana|   500|complete|
|    1009|   Diana|   250|complete|
+--------+--------+------+--------+



In [17]:
df_remove_duplicates = df_remove_null.dropDuplicates()
df_remove_duplicates.show(19)

+--------+--------+------+--------+
|order_id|customer|amount|  status|
+--------+--------+------+--------+
|    1001|   Alice|   200|complete|
|    1003|   Alice|   350|complete|
|    1008|   Diana|   500|complete|
|    1005|     Bob|   300|complete|
|    1007| Charlie|   200| pending|
|    1009|   Diana|   250|complete|
|    1004| Charlie|   150|complete|
+--------+--------+------+--------+



In [20]:
completed_orders = df_remove_duplicates.filter(df_remove_duplicates["status"] =='complete')
completed_orders.show()

+--------+--------+------+--------+
|order_id|customer|amount|  status|
+--------+--------+------+--------+
|    1001|   Alice|   200|complete|
|    1003|   Alice|   350|complete|
|    1008|   Diana|   500|complete|
|    1005|     Bob|   300|complete|
|    1009|   Diana|   250|complete|
|    1004| Charlie|   150|complete|
+--------+--------+------+--------+



In [21]:
grp_data = completed_orders.groupBy("customer").sum("amount");
grp_data.show()

+--------+-----------+
|customer|sum(amount)|
+--------+-----------+
|   Diana|        750|
| Charlie|        150|
|     Bob|        300|
|   Alice|        550|
+--------+-----------+



In [22]:
data_sorted = grp_data.orderBy('sum(amount)',ascending = False).show()

+--------+-----------+
|customer|sum(amount)|
+--------+-----------+
|   Diana|        750|
|   Alice|        550|
|     Bob|        300|
| Charlie|        150|
+--------+-----------+

